In [ ]:
import numpy as np
import pandas as pd
import matplotlib as plt

import json
from io import StringIO

import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.pricing as pricing

from oandapyV20.contrib.factories import InstrumentsCandlesFactory

accountID = "101-004-11825438-001"
access_token = "1f5b26a4026a2c558d1b7242d0893ec7-d8915fe936ed95e499e9d4efd57bb282"

client = oandapyV20.API(access_token=access_token)

In [ ]:
instrument = "EUR_USD"
granularity = "S5"
_from = "2020-02-05T00:00:00Z"
_to = "2020-03-05T00:00:00Z"
count = "8000"

params = {
    "from": _from,
    "to": _to,
    "granularity": granularity,
    }

In [ ]:
data = []

for r in InstrumentsCandlesFactory(instrument=instrument, params=params):
    client.request(r)
    for i in r.response['candles']:
        data.append([i['time'], i['volume'], i['mid']['o'], i['mid']['h'], i['mid']['l'], i['mid']['c']])

In [ ]:
df = pd.DataFrame(data)
df.columns = ['Time', 'Volume', 'Open', 'High', 'Low', 'Close']
df = df.set_index('Time')
df = df.astype(np.float64)
df.head()
df.info()

In [ ]:
df.to_csv('data.csv')

In [ ]:
pd.read_csv('data.csv')  

In [ ]:
df['returns'] = np.log(df['Close'] / df['Close'].shift(1))

cols = []

for momentum in [15, 30, 60, 120]:
    col = 'position_%s' % momentum 
    df[col] = np.sign(df['returns'].rolling(momentum).mean())
    cols.append(col)

In [ ]:
%matplotlib inline
import seaborn as sns; sns.set()

strats = ['returns']

for col in cols:
    strat = 'strategy_%s' % col.split('_')[1]
    df[strat] = df[col].shift(1) * df['returns']
    strats.append(strat)

df[strats].dropna().cumsum().apply(np.exp).plot()

In [ ]:
r = positions.OpenPositions(accountID=accountID)


print("REQUEST:{}".format(r))
rv = api.request(r)
print("RESPONSE:\n{}".format(json.dumps(rv, indent=4)))

print(len(r.response))

In [ ]:
data = {
  "order": {
    "price": "1.2",
    "stopLossOnFill": {
      "timeInForce": "GTC",
      "price": "1.25"
    },
    "timeInForce": "GTC",
    "instrument": "EUR_USD",
    "units": "-100",
    "type": "LIMIT",
    "positionFill": "DEFAULT"
  }
}

r = orders.OrderCreate(accountID, data=data)
print("REQUEST:{}".format(r))
rv = api.request(r)
print("RESPONSE:\n{}".format(json.dumps(rv, indent=4)))

In [ ]:
r = orders.OrderList(accountID)
client.request(r)
print("REQUEST:{}".format(r))
rv = api.request(r)
print("RESPONSE:\n{}".format(json.dumps(rv, indent=4)))

In [ ]:
api = API(access_token=access_token)


r = trades.TradesList(accountID)

print("REQUEST:{}".format(r))
rv = api.request(r)
print("RESPONSE:\n{}".format(json.dumps(rv, indent=4)))

In [ ]:
r = orders.OrderList(accountID)
print("REQUEST:{}".format(r))
rv = api.request(r)
print("RESPONSE:\n{}".format(json.dumps(rv, indent=4)))

print(len(r.response))

In [ ]:
params ={
          "instruments": "EUR_USD"}
        
r = pricing.PricingInfo(accountID=accountID, params=params)

print("REQUEST:{}".format(r))
rv = api.request(r)
print("RESPONSE:\n{}".format(json.dumps(rv, indent=4)))

In [ ]:
import sklearn